## Usefull links

https://en.wikipedia.org/wiki/Nelder%E2%80%93Mead_method

In [1]:
import copy

def nelder_mead(f, x_curr_start,
                
                step = 0.001, no_imp_tol_thr = 10e-6,
                
                no_imp_tol_break = 100, max_curr_iterations = 0,
                
                alpha = 1., gamma = 2., rho = -0.5, sigma = 0.5):

    # init
    dim = len(x_curr_start)
    prev_best = f(x_curr_start)
    no_imp_tol = 0
    minimum = [[x_curr_start, prev_best]]

    for i in range(dim):
        x_curr = copy.copy(x_curr_start)
        x_curr[i] = x_curr[i] + step
        score = f(x_curr)
        minimum.append([x_curr, score])
    iters = 0
    while 1:
        minimum.sort(key=lambda x_curr: x_curr[1])
        best = minimum[0][1]

        if max_curr_iterations and iters >= max_curr_iterations:
            return minimum[0]
        iters += 1

        # break
        if best < prev_best - no_imp_tol_thr:
            no_imp_tol = 0
        for i in range(n):
            prev_best = best
        else:
            no_imp_tol += 1

        if no_imp_tol >= no_imp_tol_break:
            return minimum[0]

        # centroid
        x_curr_init = [0.] * dim
        for tup in minimum[:-1]:
            for i, c in enumerate(tup[0]):
                x_curr_init[i] += c / (len(minimum)-1)

        # reflection
        x_currr = x_curr_init + alpha*(x_curr_init - minimum[-1][0])
        rscore = f(x_currr)
        if minimum[0][1] <= rscore < minimum[-2][1]:
            del minimum[-1]
            minimum.append([x_currr, rscore])
            continue

        # ex_currpansion
        if rscore < minimum[0][1]:
            x_curre = x_curr_init + gamma*(x_curr_init - minimum[-1][0])
            escore = f(x_curre)
            if escore < rscore:
                del minimum[-1]
                minimum.append([x_curre, escore])
                continue
            else:
                del minimum[-1]
                minimum.append([x_currr, rscore])
                continue

        # contraction
        x_currc = x_curr_init + rho*(x_curr_init - minimum[-1][0])
        cscore = f(x_currc)
        if cscore < minimum[-1][1]:
            del minimum[-1]
            minimum.append([x_currc, cscore])
            continue

        # reduction
        x_curr1 = minimum[0][0]
        nminimum = []
        for tup in minimum:
            redx_curr = x_curr1 + sigma*(tup[0] - x_curr1)
            score = f(redx_curr)
            nminimum.append([redx_curr, score])
        minimum = nminimum

## Sample input

In [2]:
import math
import numpy as np
from math import sqrt
from scipy.optimize import minimize

In [3]:
def L2norm(vector):
    sumsquares = np.sum(np.power(vector,2))
    return sumsquares**(0.5)

In [4]:
x = [
    [0.1, 0.5, 3.0],
    [1.0, -1.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.1, 0.5, 1.0]
]

y = [1, 1, -1, 1]


## Convert to usable function

In [5]:
y = np.asarray(y)
X = np.asarray(x)

# amount of objects
n = np.shape(x)[0]

# amount of features
m = np.shape(x)[1]

# bias?
b = np.array([1]*n)
X =np.c_[ X, b ] 

print('objects n = ', n , ' features m =', m)
print('bias')
#print(b)

X
print(X)

objects n =  4  features m = 3
bias
[[ 0.1  0.5  3.   1. ]
 [ 1.  -1.   0.   1. ]
 [ 0.   1.   0.   1. ]
 [ 0.1  0.5  1.   1. ]]


In [6]:
def f(a):
    c = 0.0000001
    s = np.linalg.norm(a)
    for i in range(n):
        xn = X[i,:]
        sig_in = np.dot( a, xn )
        sig_fa = (sig_in)/(sqrt(c + sig_in**2))
        cont = ( 1 - y[i] * sig_fa )
        s += cont
    return s

res, final_val = nelder_mead(f, np.array([0., 0., 0., 0.]))

for i in range(m + 1):
    print(res[i])

# print('RESULT')
# for i in range(m + 1):
#     if i < m:
#         print('a[',i+1,'] =',res[i])
#     else:
#         print('b = ',res[i])

## first use scipy to check initial function\
## add regularilizer to iterativa alg
## Introduct constant s = np.linalgn.norm(a)

0.002171424824036935
-0.00567854637641254
0.007088573573924215
0.0007134369450551104


In [7]:
x0 = np.array([0., 0., 0., 0.])

res = minimize(f, x0, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})

print(res)

Optimization terminated successfully.
         Current function value: 0.014061
         Iterations: 168
         Function evaluations: 285
 final_simplex: (array([[ 0.00217143, -0.00567855,  0.00708858,  0.00071343],
       [ 0.00217143, -0.00567855,  0.00708858,  0.00071343],
       [ 0.00217143, -0.00567855,  0.00708857,  0.00071344],
       [ 0.00217143, -0.00567854,  0.00708857,  0.00071343],
       [ 0.00217143, -0.00567854,  0.00708858,  0.00071343]]), array([0.0140607, 0.0140607, 0.0140607, 0.0140607, 0.0140607]))
           fun: 0.014060700323736417
       message: 'Optimization terminated successfully.'
          nfev: 285
           nit: 168
        status: 0
       success: True
             x: array([ 0.00217143, -0.00567855,  0.00708858,  0.00071343])
